Работу выполнили:

Студенты группы 208М:  
Филатова Лада   
Ильин Илья  
Ланин Олег  

Описание:
1. Невеста ищет себе жениха (существует единственное вакантное место).
2. Число претендентов - N.
3. Невеста общается с претендентами в случайном порядке, с каждым не более одного раза.
4. О каждом претенденте известно, лучше он или хуже любого из предыдущих.
5. Пообщавшись с претендентом, невеста сравнивает его с предыдущими и либо отказывает, либо принимает его предложение. Если предложение принято, они женятся и процесс останавливается. Если невеста отказывает жениху, то вернуться к нему позже она не сможет.
6. Невеста выигрывает, если она выберет самого лучшего претендента. Выбор даже второго по порядку сравнения — проигрыш.

Задача:
1. Разработать и обучить нейронную сеть, которая возьмет на себя роль невесты и будет решать принять предложение или отказать очередному жениху.
2. Сравнить результат принятия решения обученной нейронной сети и оптимальным математическим алгоритмом.

In [1]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
import matplotlib.pyplot as plt
import random

Создание окружения BrideChoiceEnv, которое моделирует процесс выбора жениха.

In [2]:
class BrideChoiceEnv(gym.Env):
    def __init__(self, n_candidates):
        super(BrideChoiceEnv, self).__init__()
        
        self.n_candidates = n_candidates  #число претендентов
        self.candidates = np.arange(1, n_candidates + 1)  
        self.random_order = np.random.permutation(self.candidates)  
        self.best_candidate = np.max(self.candidates)  #лучший претендент 
        
        self.current_candidate_idx = 0  
        self.previous_best = -1  
        self.chosen_candidate = None  
        self.previous_candidates = []  
        

        #определение пространства действий (0 - отказать, 1 - принять)
        self.action_space = spaces.Discrete(2)
        
        #определение пространства наблюдений: нормированный номер шага и флаг, лучше ли кандидат предыдущего
        self.observation_space = spaces.Dict({
            'norm_num':spaces.Box(0, 1, dtype=np.float32),
            'is_better':spaces.Discrete(2)
        })
        
    def reset(self, seed = None): #перезапуск 
        super().reset(seed = seed)
        np.random.seed(seed)
        self.random_order = np.random.permutation(self.candidates)  
        self.current_candidate_idx = 0
        self.previous_best = -1
        self.chosen_candidate = None
        self.previous_candidates = []
        return self._get_observation(), {}
    
    def _get_observation(self): #смотрим на текущего кандидата
        normalized_step = self.current_candidate_idx / (self.n_candidates) if self.n_candidates > 1 else 0
        #лучше ли он по сравнению с предыдущим максимумом
        if self.current_candidate_idx < self.n_candidates:
            current_is_better = self.random_order[self.current_candidate_idx] >= self.previous_best
        else:
            current_is_better = 0
        return {"norm_num":np.array([normalized_step], dtype=np.float32), "is_better": int(current_is_better)}
    
    
    def step(self, action): #шаг, 
        current_candidate = self.random_order[self.current_candidate_idx]  #текущий
        done = False
        reward = 0
        
        if action == 1:  #принять кандидата
            self.chosen_candidate = current_candidate
            done = True
            if self.chosen_candidate == self.best_candidate:  #если выбран лучший претендент - победа
                reward = 1
            else:
                reward = -1 
        else:  #отвергнуть, добавляем текущего в список отвергнутых
            self.previous_candidates.append(current_candidate)
            self.previous_best = max(self.previous_best, current_candidate)
            self.current_candidate_idx += 1
            
            if self.current_candidate_idx == self.n_candidates:  #если все кандидаты просмотрены
                done = True
                if self.chosen_candidate == self.best_candidate:
                    reward = 1  #победа, если выбран лучший
                else:
                    reward = -1  #проигрыш
        
        return self._get_observation(), reward, done, False, {}
    
    def render(self): #текущее состояние
        if self.chosen_candidate is not None:
            print(f"Невеста выбрала претендента {self.chosen_candidate}.")
        else:
            print(f"Невеста продолжает искать... Претендент №{self.random_order[self.current_candidate_idx]}")


Математическая оптимальная стратегия. Основывается на правиле 1/е (37%).  
Просматриваем первые 37% кандидатов (1/е), и после этого выбираем первого лучшего, которого встретим.  
Если текущий шаг больше или равен 1/е и кандидат лучше всех предыдущих — выбрать кандидата (return 1).  
Во всех остальных случаях — отказать (return 0).  

In [3]:
env = BrideChoiceEnv(100)

# Оптимальная стратегия
def optimal_strategy(step, is_best):
    if step >= (1/np.e) and is_best:
        return 1
    else:
        return 0
    

episodes = 3000
optimal_wins = 0

for _ in range(episodes):
    obs, _ = env.reset()
    done = False
    while not done:
        cur = obs["norm_num"]
        is_best = obs["is_better"]
        action = optimal_strategy(cur, is_best)
        obs, reward, done, _, _ = env.step(action)
    if reward == 1:
        optimal_wins += 1

optimal_wins/episodes #вероятность успеха

0.36633333333333334

Нейронная сеть. Применяется алгоритм Proximal Policy Optimization (PPO) — метод обучения с подкреплением (RL).

In [4]:
env = BrideChoiceEnv(100)

model = PPO("MultiInputPolicy", env, verbose=1)

training_steps = 70_000  #шаги обучения
model.learn(total_timesteps=training_steps)



Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2.12     |
|    ep_rew_mean     | -0.98    |
| time/              |          |
|    fps             | 1059     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.44        |
|    ep_rew_mean          | -0.98       |
| time/                   |             |
|    fps                  | 784         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.021128453 |
|    clip_fraction        | 0.748       |
|    clip_range           | 0.2         |
|    entropy_loss   

На каждом шаге модель получает текущее состояние (obs), включающее:  
norm_num: нормализованный номер шага  
is_better: флаг, лучше ли текущий кандидат предыдущих  

In [5]:
env = BrideChoiceEnv(100)

episodes = 1000
model_wins = 0

for _ in range(episodes):
    obs, _ = env.reset()
    done = False
    while not done:
        action, _ = model.predict(obs) #предсказывает выбрать кандидата или продолжить поиск
        obs, reward, done, _, _ = env.step(action)
        
    if reward == 1:
        model_wins += 1

model_wins/episodes #вероятность успеха модели

0.353